In [1]:
from rmgpy.species import Species
from IPython.display import display
from rmgpy.molecule.resonance import *
from rmgpy.molecule import Molecule

In [2]:
molList = generateResonanceStructures(Molecule(SMILES="N#S"))
print "species has {0} resonance structures:\n".format(len(molList))
for struc in molList:
    print "{0}".format(struc)
    display(struc)

species has 3 resonance structures:

<Molecule "S#N">


<Molecule "[SH+]=[N-]">


<Molecule "S[N]">


In [3]:
molList = generateResonanceStructures(Molecule(SMILES="OS(=O)[O]"))
print "species has {0} resonance structures:\n".format(len(molList))
for struc in molList:
    print "{0}".format(struc)
    display(struc)

AtomTypeError: Unable to determine atom type for atom S--, which has 1 single bonds, 0 double bonds to C, 1 double bonds to O, 0 double bonds to S, 1 triple bonds, 0 benzene bonds, 1 lone pairs, and -2 charge.

In [5]:
def res(mol):
    count = 0
    isomers = generateResonanceStructures(mol)
    overallIsomers = len(isomers)
    print "species has {0} resonance structures:\n".format(overallIsomers)

    # start here when copying to resonance.py
    firstIsomerFlag = True
    span = []
    missOctetList = []
    for isomer in isomers:
        missOctet = 0
        for atom in isomer.vertices:
            if atom.isOxygen() or atom.isNitrogen() or atom.isSulfur():
                val = 2 * (int(atom.getBondOrdersForAtom()) + atom.lonePairs) + atom.radicalElectrons - atom.charge
                if val != 8:
                    missOctet += abs(8 - val)
        missOctetList.append(missOctet)
        if firstIsomerFlag:
            minMissOctet = abs(missOctet)
            firstIsomerFlag = False
        else:
            if missOctet < minMissOctet: minMissOctet = missOctet
    
    filteredIsomers = []
    i = 0
    for i in xrange(len(isomers)):
        print "{0}".format(isomers[i])
        if missOctetList[i] == minMissOctet:
            filteredIsomers.append(isomers[i])
            count += 1
            print "missOctet: {0}".format(
                missOctetList[i])
        else:
            print "missOctet: {0} (REMOVED!)".format(
                missOctetList[i])
        display(isomers[i])

    print "Passed: {0}/{1} ({2}%)".format(count,overallIsomers,100.0*count/overallIsomers)

In [ ]:
mol=Molecule(SMILES='N[NH]')

res(mol)

In [3]:
Adj="""
multiplicity 2
1 O u0 p2 c0 {2,S} {4,S}
2 N u0 p1 c0 {1,S} {3,D}
3 O u0 p2 c0 {2,D}
4 N u0 p1 c0 {1,S} {5,D}
5 N u1 p1 c0 {4,D}
"""
mol = Molecule().fromAdjacencyList(Adj)

res(mol)

NameError: name 'res' is not defined

In [ ]:
mol=Molecule(SMILES='OS(=O)[O]')

res(mol)

In [ ]:
mol=Molecule(SMILES='N#[S]')

res(mol)

In [ ]:
Adj="""
multiplicity 2
1 O u0 p2 c0 {2,S} {4,S}
2 N u0 p1 c0 {1,S} {3,D}
3 O u0 p2 c0 {2,D}
4 N u0 p1 c0 {1,S} {5,D}
5 N u1 p1 c0 {4,D}
"""
mol2=Molecule(SMILES='S#N')
mol = Molecule().fromAdjacencyList(Adj)
count = 0
molList = generateResonanceStructures(mol)
print "species has {0} resonance structures:\n".format(len(molList))
for struc in molList:
    chargeSpan = sum([abs(atom.charge) for atom in struc.vertices])
    bondSpan = ('=' in struc.toSMILES()) + ('#' in struc.toSMILES())
    if bondSpan <= 10 and chargeSpan <=20:
        print "{0}".format(struc)
        print "chargeSpan: {0}, bondSpan: {1}".format(chargeSpan,bondSpan)
        display(struc)
        count += 1
count

In [ ]:
count = 0
isomers = generateResonanceStructures(mol2)
print "species has {0} resonance structures:\n".format(len(isomers))

# start here when copying to resonance.py
firstIsomerFlag = True
span = []
for isomer in isomers:
    isomerChargeSpan = int(sum([abs(atom.charge) for atom in isomer.vertices]))
    isomerBondSpan = 2 * isomer.toSMILES().count('=') + 3 * isomer.toSMILES().count('#')
    isomerValanceSpan = 0
    for atom in isomer.vertices:
        if ((atom.isNitrogen() and atom.lonePairs >= 2)
                or (atom.isOxygen() and atom.lonePairs == 1)):
            isomerValanceSpan += 1
    if firstIsomerFlag: # initialize the min values
        minIsomerChargeSpan = isomerChargeSpan
        minIsomerBondSpan = isomerBondSpan
        minIsomerValanceSpan = isomerValanceSpan
        firstIsomerFlag = False
    else:
        if isomerChargeSpan < minIsomerChargeSpan: minIsomerChargeSpan = isomerChargeSpan
        if isomerBondSpan < minIsomerBondSpan: minIsomerBondSpan = isomerBondSpan
        if isomerValanceSpan < minIsomerValanceSpan: minIsomerValanceSpan = isomerValanceSpan
    span.append((isomerChargeSpan,isomerBondSpan,isomerValanceSpan))
filteredSpan = []
filteredIsomers = []
i = 0
for i in xrange(len(isomers)):
    added = False
    if (span[i][0] <= minIsomerChargeSpan + 1) and (span[i][1] <= minIsomerBondSpan + 2) and (span[i][2] <= minIsomerValanceSpan + 1):
        filteredSpan.append(span[i])
        filteredIsomers.append(isomers[i])
        added = True
        count += 1
    if added:
        print "{0}".format(isomers[i])
        print "isomerChargeSpan: {0}, isomerBondSpan: {1}, isomerValanceSpan: {2}".format(
            span[i][0],span[i][1],span[i][2])
    else:
        print "REMOVED: {0}".format(isomers[i])
        print "isomerChargeSpan: {0}, isomerBondSpan: {1}, isomerValanceSpan: {2} REMOVED!".format(
            span[i][0],span[i][1],span[i][2])
    display(isomers[i])

print "Passed: {0}".format(count)

In [ ]:
Adj="""
multiplicity 2
1 O u0 p2 c0 {2,S} {4,S}
2 N u0 p1 c0 {1,S} {3,D}
3 O u0 p2 c0 {2,D}
4 N u0 p1 c0 {1,S} {5,D}
5 N u1 p1 c0 {4,D}
"""
mol = Molecule().fromAdjacencyList(Adj)
for atom in mol.vertices:
    print atom

In [ ]:
mol = Molecule().fromAdjacencyList(Adj)
for atom in mol.vertices:
    print atom

In [ ]:
list = []
list.append((2,4))
list.append((4,8))
list[0][0]

In [ ]:
inf
molList = generateResonanceStructures(Molecule(SMILES="[N+]=[S-]=O"))
print "species has {0} resonance structures:\n".format(len(molList))
for struc in molList:
    print "{0}".format(struc)
    display(struc)

In [ ]:
molList = generateResonanceStructures(Molecule(SMILES="O(N=O)[N]#N"))
print "species has {0} resonance structures:\n".format(len(molList))
for struc in molList:
    print "{0}".format(struc)
    display(struc)

In [ ]:
molList = generateResonanceStructures(Molecule(SMILES="[S]O"))
print "species has {0} resonance structures:\n".format(len(molList))
for struc in molList:
    print "{0}".format(struc)
    display(struc)

In [ ]:
molList = generateResonanceStructures(Molecule(SMILES="O[S]"))
print "species has {0} resonance structures:\n".format(len(molList))
for struc in molList:
    print "{0}".format(struc)
    display(struc)

In [ ]:
spc = Species().fromSMILES("O=S=O")
display(spc)
print spc.toAdjacencyList()
spc.generateResonanceIsomers()
print "species has {0} resonance structures:\n".format(len(spc.molecule))

for struc in spc.molecule:
    print "{0}".format(struc)
    display(struc)
    


In [ ]:
molList = generateResonanceStructures(Molecule(SMILES="[N+]=[S-]=O"))
print "species has {0} resonance structures:\n".format(len(molList))
for struc in molList:
    print "{0}".format(struc)
    display(struc)

In [ ]:
spc = Species().fromSMILES("CC=S[S]")
spc.generateResonanceIsomers()
print "species has {0} resonance structures:\n".format(len(spc.molecule))

for struc in spc.molecule:
    print "{0}".format(struc)
    display(struc)

In [ ]:
spc = Species().fromAdjacencyList(
                """
multiplicity 2
1 S u1 p0 c+1 {2,S} {3,D} {4,S}
2 O u0 p2 c0 {1,S} {5,S}
3 O u0 p2 c0 {1,D}
4 O u0 p3 c-1 {1,S}
5 H u0 p0 c0 {2,S}
                """)
spc.generateResonanceIsomers()
print "species has {0} resonance structures:\n".format(len(spc.molecule))

for struc in spc.molecule:
    print "{0}".format(struc)
    display(struc)

spc.molecule[0].atoms[0].atomType

In [ ]:
spc = Species().fromAdjacencyList(
                """
                1 S u0 p0 c0 {2,D} {3,D} {4,S} {5,S}
2 O u0 p2 c0 {1,D}
3 O u0 p2 c0 {1,D}
4 O u0 p2 c0 {1,S} {6,S}
5 O u0 p2 c0 {1,S} {7,S}
6 H u0 p0 c0 {4,S}
7 H u0 p0 c0 {5,S}
                """)
spc.generateResonanceIsomers()
print "species has {0} resonance structures:\n".format(len(spc.molecule))

for struc in spc.molecule:
    print "{0}".format(struc)
    display(struc)

In [ ]:
spc = Species().fromSMILES("C[N+](=N)[O-]")
spc.generateResonanceIsomers()
print "species has {0} resonance structures:\n".format(len(spc.molecule))

for struc in spc.molecule:
    print "{0}".format(struc)
    display(struc)

In [ ]:
spc = Species().fromSMILES("C[N+]([NH-])=O")
spc.generateResonanceIsomers()
print "species has {0} resonance structures:\n".format(len(spc.molecule))

for struc in spc.molecule:
    print "{0}".format(struc)
    display(struc)

In [ ]:
spc = Species().fromSMILES("[CH2]C=O")
spc.generateResonanceIsomers()
print "species has {0} resonance structures:\n".format(len(spc.molecule))

for struc in spc.molecule:
    print "{0}".format(struc)
    display(struc)

In [ ]:
spc = Species().fromSMILES("[CH]C=O")
spc.generateResonanceIsomers()
print "species has {0} resonance structures:\n".format(len(spc.molecule))

for struc in spc.molecule:
    print "{0}".format(struc)
    display(struc)

In [ ]:
def res(mol):
    count = 0
    isomers = generateResonanceStructures(mol)
    overallIsomers = len(isomers)
    print "species has {0} resonance structures:\n".format(overallIsomers)

    # start here when copying to resonance.py
    firstIsomerFlag = True
    span = []
    missingOctet = []
    for isomer in isomers:
        isomerChargeSpan = int(sum([abs(atom.charge) for atom in isomer.vertices]))
        isomerBondSpan = 2 * isomer.toSMILES().count('=') + 3 * isomer.toSMILES().count('#')
        isomerValanceSpan = 0
        for atom in isomer.vertices:
            if atom.isNitrogen():
                isomerValanceSpan += abs(atom.lonePairs -1)
                if atom.lonePairs == 2: # extra penalty for N::
                    isomerValanceSpan += 1
            elif atom.isOxygen():
                isomerValanceSpan += abs(atom.lonePairs -2)
                #if atom.lonePairs == 1: # extra penalty for O:
                #    isomerValanceSpan += 1
            elif atom.isSulfur():
                isomerValanceSpan += abs(atom.lonePairs -2)
                if atom.lonePairs == 3: # extra penalty for S:::
                    isomerValanceSpan += 1
        isomerSpanScore = 1 + isomerChargeSpan + isomerBondSpan + isomerValanceSpan
        if firstIsomerFlag: # initialize the min values
            minIsomerSpanScore = isomerSpanScore
            firstIsomerFlag = False
        else:
            if isomerSpanScore < minIsomerSpanScore: minIsomerSpanScore = isomerSpanScore
        span.append((isomerSpanScore,isomerChargeSpan,isomerBondSpan,isomerValanceSpan))
    
    filteredSpan = []
    filteredIsomers = []
    i = 0
    for i in xrange(len(isomers)):
        added = False
        if span[i][0] <= 2 * minIsomerSpanScore:
            filteredSpan.append(span[i])
            filteredIsomers.append(isomers[i])
            added = True
            count += 1
        if added:
            print "{0}".format(isomers[i])
            print "isomerChargeSpan: {0}, isomerBondSpan: {1}, isomerValanceSpan: {2}, score: {3}".format(
                span[i][1],span[i][2],span[i][3],span[i][0])
        else:
            print "REMOVED: {0}".format(isomers[i])
            print "isomerChargeSpan: {0}, isomerBondSpan: {1}, isomerValanceSpan: {2}, score: {3} REMOVED!".format(
                span[i][1],span[i][2],span[i][3],span[i][0])
        display(isomers[i])

    print "Passed: {0}/{1} ({2}%)".format(count,overallIsomers,100.0*count/overallIsomers)